# INS/GNSS Integration: Navigation System Fusion

This notebook covers Inertial Navigation System (INS) and Global Navigation Satellite System (GNSS) integration for navigation and tracking. We explore:

1. **INS Fundamentals** - Strapdown mechanization and error sources
2. **GNSS Measurements** - Pseudorange and position solutions
3. **Loosely-Coupled Integration** - Position/velocity updates
4. **Tightly-Coupled Integration** - Pseudorange updates
5. **Dilution of Precision** - Geometry effects on accuracy

## Prerequisites

```bash
pip install nrl-tracker plotly numpy
```

In [ ]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from pytcl.navigation import (
    # Constants
    OMEGA_EARTH, A_EARTH, F_EARTH,
    # INS
    INSState, IMUData, INSErrorState,
    mechanize_ins_ned, initialize_ins_state,
    gravity_ned, earth_rate_ned, transport_rate_ned,
    radii_of_curvature, coarse_alignment,
    coning_correction, sculling_correction,
    # GNSS
    SPEED_OF_LIGHT, GPS_L1_FREQ,
    SatelliteInfo, GNSSMeasurement,
    compute_dop, compute_line_of_sight,
    satellite_elevation_azimuth,
    # Integration
    initialize_ins_gnss, loose_coupled_predict,
    loose_coupled_update, tight_coupled_update,
    # Geodesy
    geodetic_to_ecef, ecef_to_geodetic,
    ecef_to_ned, ned_to_ecef,
)
from pytcl.coordinate_systems import euler2rotmat, rotmat2euler

np.random.seed(42)

# Plotly dark theme template
dark_template = go.layout.Template()
dark_template.layout = go.Layout(
    paper_bgcolor='#0d1117',
    plot_bgcolor='#0d1117',
    font=dict(color='#e6edf3'),
    xaxis=dict(gridcolor='#30363d', zerolinecolor='#30363d'),
    yaxis=dict(gridcolor='#30363d', zerolinecolor='#30363d'),
)

## 1. INS Fundamentals

### Inertial Navigation Concept

An INS uses:
- **Accelerometers**: Measure specific force (acceleration minus gravity)
- **Gyroscopes**: Measure angular rate

By integrating these measurements:
1. Integrate angular rates → Attitude
2. Remove gravity → Acceleration
3. Integrate acceleration → Velocity
4. Integrate velocity → Position

### Error Sources

| Error Type | Effect | Typical Value |
|------------|--------|---------------|
| Gyro bias | Position error grows as $t^2$ | 0.01-1 °/hr |
| Accel bias | Position error grows as $t^2$ | 10-1000 µg |
| Gyro noise | Attitude random walk | 0.001-0.1 °/√hr |
| Accel noise | Velocity random walk | 10-100 µg/√Hz |

In [ ]:
# Initial position: Washington DC
lat0 = np.radians(38.9)
lon0 = np.radians(-77.0)
alt0 = 100.0  # meters

# Initial velocity: moving northeast at 10 m/s
v_north = 7.0
v_east = 7.0
v_down = 0.0

# Initial attitude (roll, pitch, yaw)
roll0 = np.radians(0)
pitch0 = np.radians(0)
yaw0 = np.radians(45)  # Heading NE

# Get gravity at this location
g = gravity_ned(lat0, alt0)
print(f"Gravity at lat={np.degrees(lat0):.1f}°: {g:.4f} m/s²")

# Get radii of curvature
R_M, R_N = radii_of_curvature(lat0)
print(f"Meridional radius: {R_M/1e6:.3f} million m")
print(f"Prime vertical radius: {R_N/1e6:.3f} million m")

# Earth rotation rate in NED frame
omega_ie = earth_rate_ned(lat0)
print(f"\nEarth rate in NED: {omega_ie * 3600 * np.degrees(1):.4f} °/hr")

# Transport rate due to motion over curved Earth
omega_en = transport_rate_ned(lat0, alt0, v_north, v_east)
print(f"Transport rate in NED: {omega_en * 3600 * np.degrees(1)} °/hr")

In [ ]:
# Initialize INS state
ins_state = initialize_ins_state(
    lat=lat0, lon=lon0, alt=alt0,
    v_n=v_north, v_e=v_east, v_d=v_down,
    roll=roll0, pitch=pitch0, yaw=yaw0
)

print("Initial INS State")
print("=" * 50)
print(f"Position: {np.degrees(ins_state.lat):.6f}°N, {np.degrees(ins_state.lon):.6f}°E, {ins_state.alt:.1f}m")
print(f"Velocity: N={ins_state.v_n:.2f}, E={ins_state.v_e:.2f}, D={ins_state.v_d:.2f} m/s")
euler = rotmat2euler(ins_state.C_b_n.T, 'ZYX')  # Body to nav
print(f"Attitude: Roll={np.degrees(euler[2]):.1f}°, Pitch={np.degrees(euler[1]):.1f}°, Yaw={np.degrees(euler[0]):.1f}°")

In [ ]:
# Simulate INS propagation with sensor errors
dt = 0.01  # 100 Hz IMU
duration = 60.0  # 60 seconds
n_steps = int(duration / dt)

# IMU error parameters
gyro_bias = np.array([0.01, -0.02, 0.015]) * np.pi / 180 / 3600  # rad/s
accel_bias = np.array([50, -30, 20]) * 1e-6 * 9.81  # m/s²
gyro_noise = 0.005 * np.pi / 180  # rad/s
accel_noise = 100 * 1e-6 * 9.81  # m/s²

# Storage
positions = []
velocities = []
attitudes = []
times = []

# True trajectory (for comparison)
true_positions = []
true_lat, true_lon, true_alt = lat0, lon0, alt0
true_vn, true_ve, true_vd = v_north, v_east, v_down

# Reset INS state
ins_state = initialize_ins_state(
    lat=lat0, lon=lon0, alt=alt0,
    v_n=v_north, v_e=v_east, v_d=v_down,
    roll=roll0, pitch=pitch0, yaw=yaw0
)

for i in range(n_steps):
    t = i * dt
    
    # True IMU outputs (straight line motion, no rotation)
    # Specific force = -gravity in body frame for level flight
    true_accel = ins_state.C_b_n @ np.array([0, 0, g])  # Body frame
    true_gyro = np.zeros(3)  # No rotation
    
    # Add biases and noise to simulate real IMU
    accel = true_accel + accel_bias + np.random.randn(3) * accel_noise
    gyro = true_gyro + gyro_bias + np.random.randn(3) * gyro_noise
    
    # Create IMU data
    imu = IMUData(
        accel=accel,
        gyro=gyro,
        dt=dt
    )
    
    # Mechanize (propagate INS)
    ins_state = mechanize_ins_ned(ins_state, imu)
    
    # True trajectory (perfect navigation)
    R_M, R_N = radii_of_curvature(true_lat)
    true_lat += true_vn * dt / (R_M + true_alt)
    true_lon += true_ve * dt / ((R_N + true_alt) * np.cos(true_lat))
    true_alt -= true_vd * dt
    
    # Store results
    if i % 10 == 0:  # Store every 10th point
        positions.append([ins_state.lat, ins_state.lon, ins_state.alt])
        velocities.append([ins_state.v_n, ins_state.v_e, ins_state.v_d])
        euler = rotmat2euler(ins_state.C_b_n.T, 'ZYX')
        attitudes.append(np.degrees(euler))
        times.append(t)
        true_positions.append([true_lat, true_lon, true_alt])

positions = np.array(positions)
velocities = np.array(velocities)
attitudes = np.array(attitudes)
true_positions = np.array(true_positions)

print(f"Propagated INS for {duration:.0f} seconds")
print(f"Final position error: {np.degrees(positions[-1, 0] - true_positions[-1, 0])*111000:.1f}m N, "
      f"{np.degrees(positions[-1, 1] - true_positions[-1, 1])*111000*np.cos(lat0):.1f}m E")

In [ ]:
# Visualize INS drift
fig = make_subplots(rows=2, cols=2,
                    subplot_titles=['INS Position Error (no GNSS)', 'INS Velocity Error',
                                   'INS Attitude', 'Position Error Trajectory'])

# Position error
north_error = (positions[:, 0] - true_positions[:, 0]) * (R_M + alt0)
east_error = (positions[:, 1] - true_positions[:, 1]) * (R_N + alt0) * np.cos(lat0)
fig.add_trace(
    go.Scatter(x=times, y=north_error, mode='lines', name='North',
               line=dict(color='#00d4ff', width=2)),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=times, y=east_error, mode='lines', name='East',
               line=dict(color='#ff4757', width=2)),
    row=1, col=1
)

# Velocity error
fig.add_trace(
    go.Scatter(x=times, y=velocities[:, 0] - v_north, mode='lines', name='North V',
               line=dict(color='#00d4ff', width=2), showlegend=False),
    row=1, col=2
)
fig.add_trace(
    go.Scatter(x=times, y=velocities[:, 1] - v_east, mode='lines', name='East V',
               line=dict(color='#ff4757', width=2), showlegend=False),
    row=1, col=2
)

# Attitude
fig.add_trace(
    go.Scatter(x=times, y=attitudes[:, 2], mode='lines', name='Roll',
               line=dict(color='#00ff88', width=2)),
    row=2, col=1
)
fig.add_trace(
    go.Scatter(x=times, y=attitudes[:, 1], mode='lines', name='Pitch',
               line=dict(color='#00d4ff', width=2)),
    row=2, col=1
)
fig.add_trace(
    go.Scatter(x=times, y=attitudes[:, 0], mode='lines', name='Yaw',
               line=dict(color='#ff4757', width=2)),
    row=2, col=1
)

# Trajectory
fig.add_trace(
    go.Scatter(x=east_error, y=north_error, mode='lines',
               line=dict(color='#00d4ff', width=2), name='Error path', showlegend=False),
    row=2, col=2
)
fig.add_trace(
    go.Scatter(x=[0], y=[0], mode='markers', name='Start',
               marker=dict(color='#00ff88', size=12)),
    row=2, col=2
)
fig.add_trace(
    go.Scatter(x=[east_error[-1]], y=[north_error[-1]], mode='markers', name='End',
               marker=dict(color='#ff4757', size=12, symbol='square')),
    row=2, col=2
)

fig.update_xaxes(title_text='Time (s)', row=1, col=1)
fig.update_yaxes(title_text='Position Error (m)', row=1, col=1)
fig.update_xaxes(title_text='Time (s)', row=1, col=2)
fig.update_yaxes(title_text='Velocity Error (m/s)', row=1, col=2)
fig.update_xaxes(title_text='Time (s)', row=2, col=1)
fig.update_yaxes(title_text='Angle (deg)', row=2, col=1)
fig.update_xaxes(title_text='East Error (m)', row=2, col=2)
fig.update_yaxes(title_text='North Error (m)', row=2, col=2, scaleanchor='x4', scaleratio=1)

fig.update_layout(template=dark_template, height=700)
fig.show()

## 2. GNSS Measurements and Geometry

### GNSS Measurement Model

The pseudorange measurement is:
$$\rho = r + c\cdot\delta t_{rx} + \epsilon$$

Where:
- $r$ = geometric range to satellite
- $c$ = speed of light
- $\delta t_{rx}$ = receiver clock bias
- $\epsilon$ = measurement noise + atmospheric delays

### Dilution of Precision (DOP)

| DOP Type | Meaning |
|----------|--------|
| GDOP | Geometric (3D position + time) |
| PDOP | Position (3D) |
| HDOP | Horizontal (2D) |
| VDOP | Vertical |
| TDOP | Time |

In [ ]:
# Simulate satellite constellation
# Generate satellites at various azimuths and elevations
n_sats = 8
sat_azimuths = np.linspace(0, 2*np.pi, n_sats, endpoint=False)
sat_elevations = np.array([45, 30, 60, 25, 55, 35, 50, 40]) * np.pi / 180

# Convert to ECEF positions (assume ~20,000 km altitude)
sat_altitude = 20200e3  # GPS orbit altitude
user_ecef = geodetic_to_ecef(lat0, lon0, alt0)

satellites = []
for i in range(n_sats):
    # Unit vector from user to satellite in NED
    az = sat_azimuths[i]
    el = sat_elevations[i]
    
    # NED unit vector
    ned_unit = np.array([
        np.cos(el) * np.cos(az),  # North
        np.cos(el) * np.sin(az),  # East
        -np.sin(el)               # Down (negative because sat is up)
    ])
    
    # Convert to ECEF
    ecef_unit = ned_to_ecef(ned_unit, lat0, lon0)
    sat_ecef = user_ecef + sat_altitude * ecef_unit
    
    satellites.append(SatelliteInfo(
        prn=i+1,
        x=sat_ecef[0],
        y=sat_ecef[1],
        z=sat_ecef[2],
        vx=0.0, vy=0.0, vz=0.0,  # Stationary for simplicity
        clock_bias=0.0,
        clock_drift=0.0
    ))

print(f"Simulated {n_sats} GPS satellites")
print("\nSatellite visibility:")
for sat in satellites:
    el, az = satellite_elevation_azimuth(
        lat0, lon0, alt0,
        sat.x, sat.y, sat.z
    )
    print(f"  PRN {sat.prn:2d}: El={np.degrees(el):5.1f}°, Az={np.degrees(az):6.1f}°")

In [ ]:
# Compute DOP values
gdop, pdop, hdop, vdop, tdop = compute_dop(lat0, lon0, alt0, satellites)

print("Dilution of Precision")
print("=" * 40)
print(f"GDOP (Geometric):   {gdop:.2f}")
print(f"PDOP (Position):    {pdop:.2f}")
print(f"HDOP (Horizontal):  {hdop:.2f}")
print(f"VDOP (Vertical):    {vdop:.2f}")
print(f"TDOP (Time):        {tdop:.2f}")

# Position accuracy estimate
sigma_uere = 3.0  # User Equivalent Range Error (m)
print(f"\nPosition Accuracy (1σ) with UERE={sigma_uere}m:")
print(f"  Horizontal: {hdop * sigma_uere:.1f} m")
print(f"  Vertical:   {vdop * sigma_uere:.1f} m")
print(f"  3D:         {pdop * sigma_uere:.1f} m")

In [ ]:
# Visualize satellite geometry
fig = make_subplots(rows=1, cols=2, 
                    specs=[[{'type': 'polar'}, {'type': 'scene'}]],
                    subplot_titles=[f'Sky Plot (HDOP={hdop:.2f}, VDOP={vdop:.2f})', 
                                   'Satellite Geometry'])

# Sky plot
for sat in satellites:
    el, az = satellite_elevation_azimuth(lat0, lon0, alt0, sat.x, sat.y, sat.z)
    r = 90 - np.degrees(el)  # Elevation inverted for polar plot
    fig.add_trace(
        go.Scatterpolar(r=[r], theta=[np.degrees(az)], mode='markers+text',
                        marker=dict(size=12), text=[str(sat.prn)],
                        textposition='top center', name=f'PRN {sat.prn}'),
        row=1, col=1
    )

# 3D visualization - Earth surface
u = np.linspace(0, 2 * np.pi, 30)
v = np.linspace(0, np.pi, 20)
r_earth = A_EARTH / 1e6
x_earth = r_earth * np.outer(np.cos(u), np.sin(v))
y_earth = r_earth * np.outer(np.sin(u), np.sin(v))
z_earth = r_earth * np.outer(np.ones(np.size(u)), np.cos(v)) * (1 - F_EARTH)

fig.add_trace(
    go.Surface(x=x_earth, y=y_earth, z=z_earth,
               colorscale=[[0, 'lightblue'], [1, 'lightblue']],
               opacity=0.3, showscale=False),
    row=1, col=2
)

# User position
fig.add_trace(
    go.Scatter3d(x=[user_ecef[0]/1e6], y=[user_ecef[1]/1e6], z=[user_ecef[2]/1e6],
                 mode='markers', marker=dict(color='#ff4757', size=8, symbol='diamond'),
                 name='User'),
    row=1, col=2
)

# Satellites
for sat in satellites:
    fig.add_trace(
        go.Scatter3d(x=[sat.x/1e6], y=[sat.y/1e6], z=[sat.z/1e6],
                     mode='markers', marker=dict(color='#00d4ff', size=5),
                     showlegend=False),
        row=1, col=2
    )
    # Line to user
    fig.add_trace(
        go.Scatter3d(x=[user_ecef[0]/1e6, sat.x/1e6],
                     y=[user_ecef[1]/1e6, sat.y/1e6],
                     z=[user_ecef[2]/1e6, sat.z/1e6],
                     mode='lines', line=dict(color='gray', width=1),
                     opacity=0.3, showlegend=False),
        row=1, col=2
    )

fig.update_layout(
    template=dark_template,
    height=500,
    polar=dict(
        radialaxis=dict(range=[0, 90], tickvals=[0, 30, 60, 90], 
                        ticktext=['90°', '60°', '30°', '0°']),
        angularaxis=dict(direction='clockwise', rotation=90)
    ),
    scene=dict(
        xaxis_title='X (million m)',
        yaxis_title='Y (million m)',
        zaxis_title='Z (million m)',
        xaxis=dict(gridcolor='#30363d'),
        yaxis=dict(gridcolor='#30363d'),
        zaxis=dict(gridcolor='#30363d'),
    )
)
fig.show()

## 3. Loosely-Coupled Integration

In loosely-coupled integration:
- GNSS provides position/velocity solutions
- INS provides attitude and high-rate navigation
- Kalman filter combines both

### Error State Model

The error state vector is:
$$\delta\mathbf{x} = [\delta\phi_n, \delta\phi_e, \delta\phi_d, \delta v_n, \delta v_e, \delta v_d, \delta\text{lat}, \delta\text{lon}, \delta\text{alt}, b_{g_x}, b_{g_y}, b_{g_z}, b_{a_x}, b_{a_y}, b_{a_z}]^T$$

In [ ]:
# Simulate integrated INS/GNSS navigation
dt_imu = 0.01  # 100 Hz IMU
dt_gnss = 1.0  # 1 Hz GNSS
duration = 120.0  # 2 minutes

n_imu_steps = int(duration / dt_imu)
gnss_interval = int(dt_gnss / dt_imu)

# Error state covariance
P = np.diag([
    np.radians(1)**2, np.radians(1)**2, np.radians(1)**2,  # Attitude (rad²)
    0.1**2, 0.1**2, 0.1**2,  # Velocity (m/s)²
    (10/R_M)**2, (10/R_N/np.cos(lat0))**2, 10**2,  # Position (rad², m²)
    (0.01*np.pi/180/3600)**2,  # Gyro bias x
    (0.01*np.pi/180/3600)**2,  # Gyro bias y
    (0.01*np.pi/180/3600)**2,  # Gyro bias z
    (100e-6*9.81)**2,  # Accel bias x
    (100e-6*9.81)**2,  # Accel bias y
    (100e-6*9.81)**2,  # Accel bias z
])

# Process noise
Q = np.diag([
    (0.001*np.pi/180)**2,  # Gyro noise
    (0.001*np.pi/180)**2,
    (0.001*np.pi/180)**2,
    (100e-6*9.81)**2,  # Accel noise
    (100e-6*9.81)**2,
    (100e-6*9.81)**2,
    1e-20, 1e-20, 1e-20,  # Position (small)
    (0.0001*np.pi/180/3600)**2,  # Gyro bias stability
    (0.0001*np.pi/180/3600)**2,
    (0.0001*np.pi/180/3600)**2,
    (1e-6*9.81)**2,  # Accel bias stability
    (1e-6*9.81)**2,
    (1e-6*9.81)**2,
]) * dt_imu

# GNSS measurement noise
R_gnss = np.diag([
    (3.0/R_M)**2,  # Latitude (3m)
    (3.0/R_N/np.cos(lat0))**2,  # Longitude (3m)
    5.0**2,  # Altitude (5m)
    0.1**2, 0.1**2, 0.1**2,  # Velocity (0.1 m/s)
])

print("Filter initialized")
print(f"State dimension: {len(P)}")
print(f"GNSS update rate: {1/dt_gnss:.0f} Hz")
print(f"IMU rate: {1/dt_imu:.0f} Hz")

In [ ]:
# Run integrated navigation
ins_state = initialize_ins_state(
    lat=lat0, lon=lon0, alt=alt0,
    v_n=v_north, v_e=v_east, v_d=v_down,
    roll=roll0, pitch=pitch0, yaw=yaw0
)

# Error state
error_state = np.zeros(15)

# Storage
integrated_positions = []
ins_only_positions = []
gnss_positions = []
covariance_trace = []
times = []

# True trajectory
true_lat, true_lon, true_alt = lat0, lon0, alt0
true_vn, true_ve, true_vd = v_north, v_east, v_down

# INS-only state for comparison
ins_only_state = initialize_ins_state(
    lat=lat0, lon=lon0, alt=alt0,
    v_n=v_north, v_e=v_east, v_d=v_down,
    roll=roll0, pitch=pitch0, yaw=yaw0
)

for i in range(n_imu_steps):
    t = i * dt_imu
    
    # Simulate true trajectory (constant velocity)
    R_M_t, R_N_t = radii_of_curvature(true_lat)
    true_lat += true_vn * dt_imu / (R_M_t + true_alt)
    true_lon += true_ve * dt_imu / ((R_N_t + true_alt) * np.cos(true_lat))
    true_alt -= true_vd * dt_imu
    
    # Generate IMU data with errors
    true_accel = ins_state.C_b_n @ np.array([0, 0, g])
    true_gyro = np.zeros(3)
    accel = true_accel + accel_bias + np.random.randn(3) * accel_noise
    gyro = true_gyro + gyro_bias + np.random.randn(3) * gyro_noise
    
    imu = IMUData(accel=accel, gyro=gyro, dt=dt_imu)
    
    # INS mechanization
    ins_state = mechanize_ins_ned(ins_state, imu)
    ins_only_state = mechanize_ins_ned(ins_only_state, imu)
    
    # Predict covariance
    P = P + Q  # Simplified prediction
    
    # GNSS update
    if i > 0 and i % gnss_interval == 0:
        # Simulate GNSS measurement with noise
        gnss_lat = true_lat + np.random.randn() * 3.0 / R_M_t
        gnss_lon = true_lon + np.random.randn() * 3.0 / (R_N_t * np.cos(true_lat))
        gnss_alt = true_alt + np.random.randn() * 5.0
        gnss_vn = true_vn + np.random.randn() * 0.1
        gnss_ve = true_ve + np.random.randn() * 0.1
        gnss_vd = true_vd + np.random.randn() * 0.1
        
        # Innovation
        z = np.array([
            gnss_lat - ins_state.lat,
            gnss_lon - ins_state.lon,
            gnss_alt - ins_state.alt,
            gnss_vn - ins_state.v_n,
            gnss_ve - ins_state.v_e,
            gnss_vd - ins_state.v_d,
        ])
        
        # Measurement matrix (simplified)
        H = np.zeros((6, 15))
        H[0, 6] = 1  # lat
        H[1, 7] = 1  # lon
        H[2, 8] = 1  # alt
        H[3, 3] = 1  # v_n
        H[4, 4] = 1  # v_e
        H[5, 5] = 1  # v_d
        
        # Kalman gain
        S = H @ P @ H.T + R_gnss
        K = P @ H.T @ np.linalg.inv(S)
        
        # Update error state
        error_state = error_state + K @ z
        P = (np.eye(15) - K @ H) @ P
        
        # Apply corrections to INS state
        ins_state = INSState(
            lat=ins_state.lat + error_state[6],
            lon=ins_state.lon + error_state[7],
            alt=ins_state.alt + error_state[8],
            v_n=ins_state.v_n + error_state[3],
            v_e=ins_state.v_e + error_state[4],
            v_d=ins_state.v_d + error_state[5],
            C_b_n=ins_state.C_b_n,  # Attitude correction simplified
        )
        
        # Reset error state
        error_state = np.zeros(15)
        
        gnss_positions.append([gnss_lat, gnss_lon, gnss_alt])
    
    # Store results
    if i % 10 == 0:
        integrated_positions.append([ins_state.lat, ins_state.lon, ins_state.alt])
        ins_only_positions.append([ins_only_state.lat, ins_only_state.lon, ins_only_state.alt])
        covariance_trace.append(np.trace(P))
        times.append(t)

integrated_positions = np.array(integrated_positions)
ins_only_positions = np.array(ins_only_positions)
gnss_positions = np.array(gnss_positions)

print(f"\nNavigation complete: {duration:.0f} seconds")

In [ ]:
# Visualize integration results
fig = make_subplots(rows=2, cols=2,
                    subplot_titles=['Trajectory Comparison', 'Position Error Comparison',
                                   'Filter Covariance', 'Final Position Error Distribution'])

# Position comparison
true_north = np.array(times) * v_north
true_east = np.array(times) * v_east

integrated_north = (integrated_positions[:, 0] - lat0) * (R_M + alt0)
integrated_east = (integrated_positions[:, 1] - lon0) * (R_N + alt0) * np.cos(lat0)
ins_only_north = (ins_only_positions[:, 0] - lat0) * (R_M + alt0)
ins_only_east = (ins_only_positions[:, 1] - lon0) * (R_N + alt0) * np.cos(lat0)

fig.add_trace(
    go.Scatter(x=true_east, y=true_north, mode='lines', name='True',
               line=dict(color='#00ff88', width=2)),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=ins_only_east, y=ins_only_north, mode='lines', name='INS Only',
               line=dict(color='#ff4757', width=1, dash='dash'), opacity=0.7),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=integrated_east, y=integrated_north, mode='lines', name='INS/GNSS',
               line=dict(color='#00d4ff', width=1.5)),
    row=1, col=1
)
if len(gnss_positions) > 0:
    gnss_north = (gnss_positions[:, 0] - lat0) * (R_M + alt0)
    gnss_east = (gnss_positions[:, 1] - lon0) * (R_N + alt0) * np.cos(lat0)
    fig.add_trace(
        go.Scatter(x=gnss_east, y=gnss_north, mode='markers', name='GNSS',
                   marker=dict(color='#ffb800', size=4, opacity=0.5)),
        row=1, col=1
    )

# Position error
int_error_n = integrated_north - true_north
int_error_e = integrated_east - true_east
ins_error_n = ins_only_north - true_north
ins_error_e = ins_only_east - true_east

fig.add_trace(
    go.Scatter(x=times, y=np.sqrt(int_error_n**2 + int_error_e**2), mode='lines',
               name='INS/GNSS', line=dict(color='#00d4ff', width=2), showlegend=False),
    row=1, col=2
)
fig.add_trace(
    go.Scatter(x=times, y=np.sqrt(ins_error_n**2 + ins_error_e**2), mode='lines',
               name='INS Only', line=dict(color='#ff4757', width=1, dash='dash'), 
               opacity=0.7, showlegend=False),
    row=1, col=2
)

# Covariance
fig.add_trace(
    go.Scatter(x=times, y=covariance_trace, mode='lines',
               line=dict(color='#00d4ff', width=2), showlegend=False),
    row=2, col=1
)

# Error histogram
final_errors = np.sqrt(int_error_n[-100:]**2 + int_error_e[-100:]**2)
fig.add_trace(
    go.Histogram(x=final_errors, nbinsx=20, name='Error dist',
                 marker=dict(color='#00d4ff'), showlegend=False),
    row=2, col=2
)
fig.add_vline(x=np.mean(final_errors), line=dict(color='#ff4757', dash='dash'),
              annotation_text=f'Mean: {np.mean(final_errors):.2f}m', row=2, col=2)

fig.update_xaxes(title_text='East (m)', row=1, col=1)
fig.update_yaxes(title_text='North (m)', row=1, col=1, scaleanchor='x', scaleratio=1)
fig.update_xaxes(title_text='Time (s)', row=1, col=2)
fig.update_yaxes(title_text='Horizontal Error (m)', row=1, col=2)
fig.update_xaxes(title_text='Time (s)', row=2, col=1)
fig.update_yaxes(title_text='Covariance Trace', type='log', row=2, col=1)
fig.update_xaxes(title_text='Position Error (m)', row=2, col=2)
fig.update_yaxes(title_text='Count', row=2, col=2)

fig.update_layout(template=dark_template, height=700)
fig.show()

print(f"Final INS-only error: {np.sqrt(ins_error_n[-1]**2 + ins_error_e[-1]**2):.1f} m")
print(f"Final INS/GNSS error: {np.sqrt(int_error_n[-1]**2 + int_error_e[-1]**2):.1f} m")

## 4. GNSS Outage Handling

A key advantage of INS/GNSS integration is graceful degradation during GNSS outages.

In [ ]:
# Simulate navigation with GNSS outage
outage_start = 30.0  # seconds
outage_duration = 30.0  # 30 second outage

# Re-run with outage
ins_state = initialize_ins_state(
    lat=lat0, lon=lon0, alt=alt0,
    v_n=v_north, v_e=v_east, v_d=v_down,
    roll=roll0, pitch=pitch0, yaw=yaw0
)

P = np.diag([
    np.radians(1)**2, np.radians(1)**2, np.radians(1)**2,
    0.1**2, 0.1**2, 0.1**2,
    (10/R_M)**2, (10/R_N/np.cos(lat0))**2, 10**2,
    (0.01*np.pi/180/3600)**2, (0.01*np.pi/180/3600)**2, (0.01*np.pi/180/3600)**2,
    (100e-6*9.81)**2, (100e-6*9.81)**2, (100e-6*9.81)**2,
])

error_state = np.zeros(15)

outage_positions = []
outage_cov = []
outage_times = []
gnss_available = []

true_lat, true_lon, true_alt = lat0, lon0, alt0

for i in range(n_imu_steps):
    t = i * dt_imu
    
    # True trajectory
    R_M_t, R_N_t = radii_of_curvature(true_lat)
    true_lat += true_vn * dt_imu / (R_M_t + true_alt)
    true_lon += true_ve * dt_imu / ((R_N_t + true_alt) * np.cos(true_lat))
    
    # IMU data
    true_accel = ins_state.C_b_n @ np.array([0, 0, g])
    accel = true_accel + accel_bias + np.random.randn(3) * accel_noise
    gyro = gyro_bias + np.random.randn(3) * gyro_noise
    imu = IMUData(accel=accel, gyro=gyro, dt=dt_imu)
    
    # INS mechanization
    ins_state = mechanize_ins_ned(ins_state, imu)
    P = P + Q
    
    # GNSS update (with outage)
    gnss_ok = not (outage_start <= t < outage_start + outage_duration)
    
    if i > 0 and i % gnss_interval == 0 and gnss_ok:
        gnss_lat = true_lat + np.random.randn() * 3.0 / R_M_t
        gnss_lon = true_lon + np.random.randn() * 3.0 / (R_N_t * np.cos(true_lat))
        gnss_alt = true_alt + np.random.randn() * 5.0
        
        z = np.array([
            gnss_lat - ins_state.lat,
            gnss_lon - ins_state.lon,
            gnss_alt - ins_state.alt,
            true_vn + np.random.randn() * 0.1 - ins_state.v_n,
            true_ve + np.random.randn() * 0.1 - ins_state.v_e,
            np.random.randn() * 0.1 - ins_state.v_d,
        ])
        
        H = np.zeros((6, 15))
        H[0, 6] = 1; H[1, 7] = 1; H[2, 8] = 1
        H[3, 3] = 1; H[4, 4] = 1; H[5, 5] = 1
        
        S = H @ P @ H.T + R_gnss
        K = P @ H.T @ np.linalg.inv(S)
        error_state = error_state + K @ z
        P = (np.eye(15) - K @ H) @ P
        
        ins_state = INSState(
            lat=ins_state.lat + error_state[6],
            lon=ins_state.lon + error_state[7],
            alt=ins_state.alt + error_state[8],
            v_n=ins_state.v_n + error_state[3],
            v_e=ins_state.v_e + error_state[4],
            v_d=ins_state.v_d + error_state[5],
            C_b_n=ins_state.C_b_n,
        )
        error_state = np.zeros(15)
    
    if i % 10 == 0:
        outage_positions.append([ins_state.lat, ins_state.lon, ins_state.alt])
        outage_cov.append(np.sqrt(P[6, 6]) * R_M)  # Position uncertainty in meters
        outage_times.append(t)
        gnss_available.append(gnss_ok)

outage_positions = np.array(outage_positions)
outage_cov = np.array(outage_cov)
gnss_available = np.array(gnss_available)

In [ ]:
# Visualize outage behavior
fig = make_subplots(rows=2, cols=1, 
                    subplot_titles=['Navigation Error During GNSS Outage', 
                                   'Trajectory During GNSS Outage'],
                    vertical_spacing=0.12)

# Position error during outage
outage_north = (outage_positions[:, 0] - lat0) * (R_M + alt0)
outage_east = (outage_positions[:, 1] - lon0) * (R_N + alt0) * np.cos(lat0)
true_north = np.array(outage_times) * v_north
true_east = np.array(outage_times) * v_east

error_north = outage_north - true_north
error_east = outage_east - true_east
total_error = np.sqrt(error_north**2 + error_east**2)

# Outage shading
outage_times_arr = np.array(outage_times)
outage_mask = ~gnss_available
if np.any(outage_mask):
    # Find outage start and end indices
    outage_indices = np.where(outage_mask)[0]
    if len(outage_indices) > 0:
        fig.add_vrect(x0=outage_times_arr[outage_indices[0]], 
                      x1=outage_times_arr[outage_indices[-1]],
                      fillcolor='rgba(255, 71, 87, 0.2)', line_width=0,
                      row=1, col=1)

fig.add_trace(
    go.Scatter(x=outage_times, y=total_error, mode='lines', name='Position Error',
               line=dict(color='#00d4ff', width=2)),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=outage_times, y=outage_cov, mode='lines', name='1σ Uncertainty',
               line=dict(color='#00ff88', width=1.5, dash='dash')),
    row=1, col=1
)

# Trajectory with color-coded GNSS availability
fig.add_trace(
    go.Scatter(x=true_east, y=true_north, mode='lines', name='True',
               line=dict(color='#00ff88', width=2)),
    row=2, col=1
)

# Split trajectory by GNSS availability
gnss_avail_x, gnss_avail_y = [], []
gnss_outage_x, gnss_outage_y = [], []

for i in range(len(outage_times)):
    if gnss_available[i]:
        gnss_avail_x.append(outage_east[i])
        gnss_avail_y.append(outage_north[i])
    else:
        gnss_outage_x.append(outage_east[i])
        gnss_outage_y.append(outage_north[i])

fig.add_trace(
    go.Scatter(x=gnss_avail_x, y=gnss_avail_y, mode='markers', name='GNSS Available',
               marker=dict(color='#00d4ff', size=3)),
    row=2, col=1
)
fig.add_trace(
    go.Scatter(x=gnss_outage_x, y=gnss_outage_y, mode='markers', name='GNSS Outage',
               marker=dict(color='#ff4757', size=3)),
    row=2, col=1
)

fig.update_xaxes(title_text='Time (s)', row=1, col=1, range=[0, duration])
fig.update_yaxes(title_text='Error (m)', row=1, col=1)
fig.update_xaxes(title_text='East (m)', row=2, col=1)
fig.update_yaxes(title_text='North (m)', row=2, col=1, scaleanchor='x2', scaleratio=1)

fig.update_layout(template=dark_template, height=700)
fig.show()

# Find max error during outage
max_outage_error = np.max(total_error[outage_mask])
print(f"\nMax error during {outage_duration:.0f}s outage: {max_outage_error:.1f} m")
print(f"Error growth rate: {max_outage_error/outage_duration:.2f} m/s")

## Summary

Key takeaways:

1. **INS provides continuous navigation** but drifts over time due to sensor errors
2. **GNSS provides absolute position** but at lower rate and may be unavailable
3. **Loosely-coupled integration** fuses position/velocity from GNSS
4. **Tightly-coupled integration** uses raw pseudoranges for better performance
5. **DOP affects GNSS accuracy** - geometry matters!

### Integration Trade-offs

| Aspect | Loosely-Coupled | Tightly-Coupled |
|--------|----------------|----------------|
| Implementation | Simpler | Complex |
| Min satellites | 4 | 1 |
| Outage performance | Good | Better |
| Urban canyon | Poor | Better |

## Exercises

1. Implement tightly-coupled integration using pseudoranges
2. Add GNSS fault detection and exclusion (RAIM)
3. Model different IMU grades and compare performance
4. Simulate urban canyon scenarios with satellite blockage

## References

1. Groves, P. D. (2013). *Principles of GNSS, Inertial, and Multisensor Integrated Navigation Systems*.
2. Titterton, D., & Weston, J. (2004). *Strapdown Inertial Navigation Technology*.
3. Kaplan, E. D., & Hegarty, C. J. (2017). *Understanding GPS/GNSS*.